In [1]:
from lxml import html
import csv
import pandas as pd
from requests import get
import requests
from bs4 import BeautifulSoup as bs
from bs4 import SoupStrainer
import pprint

In [11]:
url = []
for i in range(1,12):
    url.append('https://careers.key.com/en-US/search?pagenumber={0}'.format(i))
print(url)

['https://careers.key.com/en-US/search?pagenumber=1', 'https://careers.key.com/en-US/search?pagenumber=2', 'https://careers.key.com/en-US/search?pagenumber=3', 'https://careers.key.com/en-US/search?pagenumber=4', 'https://careers.key.com/en-US/search?pagenumber=5', 'https://careers.key.com/en-US/search?pagenumber=6', 'https://careers.key.com/en-US/search?pagenumber=7', 'https://careers.key.com/en-US/search?pagenumber=8', 'https://careers.key.com/en-US/search?pagenumber=9', 'https://careers.key.com/en-US/search?pagenumber=10', 'https://careers.key.com/en-US/search?pagenumber=11']


In [18]:
job_description = []
job_location = []
job_title = []
weblinks1 = []  
for u in url:
    page = requests.get(u)
    soup = bs(page.text,'html.parser')
    container = soup.find("div", id = "job-results-page-main-content")
    jobs_container = container.find("div", class_ = "jrp-table-container")
    job = jobs_container.find_all("tr", class_="job-result")
    tags=[]
    for j in range(0,len(job)):
        tag = job[j].find("a")
        tags.append(tag.get('href'))
    #print(tags)
    weblinks=[] 
    for t in range(0,len(tags)):
        weblinks.append('https://careers.key.com'+tags[t])
        weblinks1.append('https://careers.key.com'+tags[t])
        response  = get(weblinks[t])
        html_soup = bs(response.text,'html.parser')
        #Scraping job description of all links in the weblinks
        class_check = html_soup.find("div", class_ = "jdp-job-description-card content-card")
        if(class_check != None):
            job_description.append(class_check.text)
        else:
            job_description.append(None)  
        #Scraping the job location
        check = html_soup.find('div', class_ = 'jdp-job-snapshot-card')
        textlocation = check.find('span', class_ = 'jobLocation')
        if(textlocation != None):
            job_location.append(textlocation.text)
        else:
            job_location.append(None)
        #scraping the job title
        check1 = html_soup.find('div', class_ = 'jdp-title-block-position')
        texttitle = check1.find('span', class_ = 'jdp-title-job-title')
        if(texttitle != None):
            job_title.append(texttitle.text)
        else:
            job_title.append(None)

In [27]:
print(len(weblinks))
print(len(job_description))
print(len(job_title))
print(len(job_location))

21
521
521
521


In [28]:
#CSV extraction for wordcount
import csv
import re
word_count=[]
with open ('wordcount_f.csv','r')as csvfile:
    csvreader=csv.reader(csvfile)
    fields=next(csvreader)  
    for row in csvreader:
        word_count.append(row)
print('length word_count',len(word_count))
wclist=[]       
for value in word_count:
    rowid,word=value
    wclist.append(word)
print('length wcist',len(wclist))
print(wclist)
#--------------------------------------------------------------------
#text rank dataset
word_count=[]
with open ('textrank_la.csv','r')as csvfile:
    csvreader=csv.reader(csvfile)
    fields=next(csvreader)  
    for row in csvreader:
        word_count.append(row)
print('length word_count',len(word_count))
trlist=[]       
for value in word_count:
    rowid,word=value
    trlist.append(word)
print('length trlist',len(trlist))
print(trlist)
#-----------------------------------------------------------------------
#tfidf dataset
word_count=[]
with open ('tfidf_f.csv','r')as csvfile:
    csvreader=csv.reader(csvfile)
    fields=next(csvreader)  
    for row in csvreader:
        word_count.append(row)
print('length word_count',len(word_count))
tfidflist=[]       
for value in word_count:
    rowid,word=value
    tfidflist.append(word)
print('length tfidflist',len(tfidflist))
print(tfidflist)



length word_count 100
length wcist 100
['financial', 'digital', 'services', 'economic', 'key', 'market', 'findings', 'platforms', 'bitcoin', 'payments', 'investors', 'insurance', 'technology', 'risk', 'transactions', 'asset', 'incumbents', 'regulators', 'institutions', 'capital', 'equity', 'global', 'investment', 'business', 'future', 'firms', 'solutions', 'management', 'infrastructure', 'potential', 'crowdfunding', 'cryptocurrency', 'contract', 'ecosystem', 'online', 'iot', 'transfer', 'kba', 'trade', 'saas', 'credit', 'markets', 'startup', 'loan', 'assets', 'validation', 'automated', 'role', 'lead', 'ledger', 'fintech', 'wef', 'lending', 'legal', 'costs', 'api', 'settlement', 'insurtech', 'partnerships', 'growth', 'evidence', 'loss', 'blockchain', 'regtech', 'mobile', 'development', 'underwriting', 'tokenization', 'exchange', 'government', 'experience', 'idps', 'money', 'stakeholders', 'innovation', 'network', 'funds', 'counterparty', 'retail', 'instech', 'unbanked', 'fico', 'interes

In [29]:
#print(job_description[0])
with open('keycorp_dataset.csv', 'w') as f:
    writer = csv.writer(f,delimiter=",", lineterminator="\n")
    #writer.writerow(word1)
    abc=['jobNo']+['Title']+['Location']+['Institution']+['List id (1,2,3)']
    for i in range(1,101):
        abc+=[str(i)]
    writer.writerow(abc)

In [98]:
with open('keycorp_final.csv','w') as f:
    writer = csv.writer(f,delimiter=",", lineterminator="\n")
    abc=['jobNo']+['Job Title']+['Job Description']+['Bank Name']+['Job Location']+['Web link']
    writer.writerow(abc)

In [99]:
def value(s):
    return ''.join([i if ord(i) < 128 else ' ' for i in s])
def cleanText(s):
    words=word_tokenize(s)
    wordlist=[]
    for word in words:
        result=value(word)
        wordlist.append(result)    
    y=' '.join(wordlist)
    return y

In [106]:
from nltk.tokenize import word_tokenize
empty='Invalid value'
#counter=0
#counter+=1
for i in range(500,521):
    with open('keycorp_final.csv','a') as f:
        varlength=[]
        writer = csv.writer(f,delimiter=",", lineterminator="\n")
        varlength.append(i)
        #varlength.append(counter)#Job ID
        #job title extraction
        jobtit=cleanText(job_title[i])
        varlength.append(jobtit)
        #job description extraction
        jobdesc=cleanText(job_description[i])
        varlength.append(jobdesc)
        varlength.append(['Key Corp Bank'])#Bank Name
        varlength.append(job_location[i])#Job Location    
        varlength.append(weblinks1[i])#Web Links
        writer.writerow(varlength)
    f.close

In [ ]:
wclist1=[]
tflist1=[]
tfidflist1=[]
empty='Invalid value'
for i in range(500,521):
    varlength=[]
    list_id=1
    if list_id==1:
        with open('keycorp_dataset.csv', 'a') as f:
            varlength=[]
            writer = csv.writer(f,delimiter=",", lineterminator="\n")
        #writer.writerow(word1)
            try: 
                varlength.append(i)
                varlength.append(job_title[i])
                varlength.append(job_location[i])                
                varlength.append(weblinks1[i])
                varlength.append([list_id])
                for value1 in wclist:
                    count2=re.findall(value1,job_description[i])
                    abc=str(len(count2))
                    empty='Invalid value'
                    varlength.append(abc)
                    #varlength.append(empty)
            except:
                print('1 Error')
                print(error)
                varlength.append('error')     
        #writer = csv.writer(f)
            try:
                writer.writerow(varlength)
            except UnicodeEncodeError:
                writer.writerow(empty)
            list_id+=1
        f.close
    if list_id==2:
        with open('keycorp_dataset.csv', 'a') as f:
            varlength=[]
            writer = csv.writer(f,delimiter=",", lineterminator="\n")
            try: 
                varlength.append(i)
                varlength.append(job_title[i])
                varlength.append(job_location[i])                
                varlength.append(weblinks1[i])
                varlength.append([list_id])
                for value1 in trlist:
                    count2=re.findall(value1,job_description[i])
                    abc=str(len(count2))
                    empty='Invalid value'
                    try:
                        varlength.append(abc)
                    except UnicodeEncodeError:
                        varlength.append(empty)
                        
                    errormsg='InvalidValue' 
            except:
                print('2 Error')
                print(error)
                varlength.append('error')     
            try:
                writer.writerow(varlength)
            except UnicodeEncodeError:
                writer.writerow(empty)
            list_id+=1
        f.close
    if list_id==3:
        with open('keycorp_dataset.csv', 'a') as f:
            varlength=[]
            writer = csv.writer(f,delimiter=",", lineterminator="\n")
            try: 
                varlength.append(i)
                varlength.append(job_title[i])
                varlength.append(job_location[i])                
                varlength.append(weblinks1[i])
                varlength.append([list_id])
                for value1 in tfidflist:
                    count2=re.findall(value1,job_description[i])
                    abc=str(len(count2))
                    empty='Invalid value'
                    try:
                        varlength.append(abc)
                        errormsg='InvalidValue' 
                    except UnicodeEncodeError:
                        varlength.append(empty)
                     
            except:
                print(' 3 Error')
                print(error)
                varlength.append('error')     
            try:
                writer.writerow(varlength)
            except UnicodeEncodeError:
                writer.writerow(empty)
        f.close

In [195]:
def sortkey(val):
    return val[0]
#job_location=['Arvada, CO','Syracuse, NY','Lakewood, CO']
#job_title=['Teller-Fulltime-Arvada Branches ', 'Teller-Fulltime Float-Lakewood Area ', 'Operation Leaders- Central Area ']
lenloc=len(job_location)
lentitle=len(job_title)
print('lenloc',lenloc,'lentitle',lentitle)
sorted_list=[]
if(lenloc==lentitle):
    list1=list(zip(job_location, job_title))
list1.sort(key = sortkey) 
print(list1)

lenloc 536 lentitle 536
[('Akron, OH', 'Mortgage Loan Officer - Akron/Canton, OH Area '), ('Albany, NY', 'Mortgage Loan Officer '), ('Albany, NY', 'Healthcare Mortgage Lending Officer (Physician,Dentist,Veterinarian) '), ('Albany, NY', '2019 Summer Retail Banking Intern- Albany '), ('Albany, NY', 'Business Banking Relationship Manager '), ('Albany, NY', 'Associate Relationship Mgr - Private Bank '), ('Aloha, OR', 'Personal Banker - Farmington '), ('Amelia, OH', 'Personal Banker - Amelia Branch '), ('Amherst, NY', 'Personal Banker - Snyder '), ('Amsterdam, NY', 'Licensed Relationship Banker-- Maple Ave '), ('Anchorage, AK', 'Financial Advisor '), ('Anchorage, AK', 'Teller - 20 Hours - Benson '), ('Anchorage, AK', 'Personal Banker - Benson '), ('Anchorage, AK', 'Float Teller - 20 Hours - Anchorage '), ('Anchorage, AK', 'Float Teller - 20 Hours - Anchorage '), ('Anchorage, AK', 'Teller - 20 Hours - 5th Avenue '), ('Anchorage, AK', 'Teller - 20 Hours - Benson '), ('Anchorage, AK', 'Float T

In [219]:
import collections
C = collections.Counter(list1)
list2=[]
list2=[ [k,]*v for k,v in C.items()]
print(list2)
    

[[('Akron, OH', 'Mortgage Loan Officer - Akron/Canton, OH Area ')], [('Albany, NY', 'Mortgage Loan Officer ')], [('Albany, NY', 'Healthcare Mortgage Lending Officer (Physician,Dentist,Veterinarian) ')], [('Albany, NY', '2019 Summer Retail Banking Intern- Albany ')], [('Albany, NY', 'Business Banking Relationship Manager ')], [('Albany, NY', 'Associate Relationship Mgr - Private Bank ')], [('Aloha, OR', 'Personal Banker - Farmington ')], [('Amelia, OH', 'Personal Banker - Amelia Branch ')], [('Amherst, NY', 'Personal Banker - Snyder ')], [('Amsterdam, NY', 'Licensed Relationship Banker-- Maple Ave ')], [('Anchorage, AK', 'Financial Advisor ')], [('Anchorage, AK', 'Teller - 20 Hours - Benson '), ('Anchorage, AK', 'Teller - 20 Hours - Benson ')], [('Anchorage, AK', 'Personal Banker - Benson ')], [('Anchorage, AK', 'Float Teller - 20 Hours - Anchorage '), ('Anchorage, AK', 'Float Teller - 20 Hours - Anchorage ')], [('Anchorage, AK', 'Teller - 20 Hours - 5th Avenue ')], [('Anchorage, AK', '